In [173]:
pip install pororo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [174]:
from pororo import Pororo
import pandas as pd
import numpy as np

In [175]:
Pororo.available_tasks()

"Available tasks are ['mrc', 'rc', 'qa', 'question_answering', 'machine_reading_comprehension', 'reading_comprehension', 'sentiment', 'sentiment_analysis', 'nli', 'natural_language_inference', 'inference', 'fill', 'fill_in_blank', 'fib', 'para', 'pi', 'cse', 'contextual_subword_embedding', 'similarity', 'sts', 'semantic_textual_similarity', 'sentence_similarity', 'sentvec', 'sentence_embedding', 'sentence_vector', 'se', 'inflection', 'morphological_inflection', 'g2p', 'grapheme_to_phoneme', 'grapheme_to_phoneme_conversion', 'w2v', 'wordvec', 'word2vec', 'word_vector', 'word_embedding', 'tokenize', 'tokenise', 'tokenization', 'tokenisation', 'tok', 'segmentation', 'seg', 'mt', 'machine_translation', 'translation', 'pos', 'tag', 'pos_tagging', 'tagging', 'const', 'constituency', 'constituency_parsing', 'cp', 'pg', 'collocation', 'collocate', 'col', 'word_translation', 'wt', 'summarization', 'summarisation', 'text_summarization', 'text_summarisation', 'summary', 'gec', 'review', 'review_s

In [176]:
ner = Pororo(task="ner", lang="ko")

2022-11-29 14:59:22 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [input] dictionary: 4005 types
2022-11-29 14:59:22 | INFO | pororo.models.brainbert.tasks.sequence_tagging | [label] dictionary: 41 types


In [177]:
from itertools import permutations , combinations
temp_list =  ner('Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.')
print(temp_list)

remove_temp_list = [(i[0],i[1])  for i in temp_list if i[1] != 'O' ]
#print(remove_temp_list)

temp = []
for idx , item in enumerate(combinations(remove_temp_list, 2)) :
    if item[0][1] == 'PERSON' :
        temp.append(item)

for i in temp :
    print(i)

[('Something', 'ARTIFACT'), ('〉는', 'O'), (' ', 'O'), ('조지 해리슨', 'PERSON'), ('이', 'O'), (' ', 'O'), ('쓰고', 'O'), (' ', 'O'), ('비틀즈', 'PERSON'), ('가', 'O'), (' ', 'O'), ('1969년', 'DATE'), (' ', 'O'), ('앨범', 'O'), (' ', 'O'), ('《', 'O'), ('Abbey Road', 'ARTIFACT'), ('》에', 'O'), (' ', 'O'), ('담은', 'O'), (' ', 'O'), ('노래다.', 'O')]
(('조지 해리슨', 'PERSON'), ('비틀즈', 'PERSON'))
(('조지 해리슨', 'PERSON'), ('1969년', 'DATE'))
(('조지 해리슨', 'PERSON'), ('Abbey Road', 'ARTIFACT'))
(('비틀즈', 'PERSON'), ('1969년', 'DATE'))
(('비틀즈', 'PERSON'), ('Abbey Road', 'ARTIFACT'))


In [178]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm
import pathlib
import pickle
from itertools import combinations , permutations

def num_to_label(label: np.ndarray) -> list:
    """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
    """
    origin_label = []
    with open("/opt/ml/code/dict_num_to_label.pkl", "rb") as f:
        dict_num_to_label = pickle.load(f)
    for v in label:
        origin_label.append(dict_num_to_label[v])

    return origin_label

class RE_Dataset(torch.utils.data.Dataset):
    """Dataset 구성을 위한 class."""

    def __init__(self, pair_dataset: pd.DataFrame, labels: np.ndarray):
        self.pair_dataset = pair_dataset
        self.labels = labels

    def __getitem__(self, idx: int) -> torch.Tensor:
        item = {
            key: val[idx].clone().detach() for key, val in self.pair_dataset.items()
        }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self) -> int:
        return len(self.labels)


class BaseDataLoader:
    """BaseLine DataLoader 입니다.

    Args:
        load_data :  (str): 가져올 데이터의 주소입니다.
        tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

    func :
        1)preprocessing_dataset(pd.DataFrame) : 데이터 전처리를 합니다 return pd.DataFrame
        2)tokenized_dataset(pd.DataFrame, tokenizer: AutoTokenizer) : 전처리된 데이터셋을 tokenized_dataset으로 반환 합니다
        3)get_dataset : train, valid 데이터셋을 토큰화 셋으로 변환한 뒤 RE_Dataset 형태로 반환 합니다
        4)get_test_dataset :  test 데이터셋을 토큰화 셋으로 변환한 뒤 RE_Dataset 형태로 반환 합니다
    """

    def __init__(self, data_path: pathlib.Path, tokenizer: AutoTokenizer):
        self.data_path = data_path
        self.tokenizer = tokenizer

    def preprocessing_dataset(self, dataset: pd.DataFrame) -> pd.DataFrame:
        """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
        subject_entity = []
        object_entity = []

        re_idx = []
        start = 0
        sentence = []
        label =[]
        
        sub_list = ['PERSON'] # ['PERSON','ORGANIZATION']
        obj_list = ['LOCATION'] # ['PERSON' , 'ORGANIZATION', 'ARTIFACT', 'DATE' , 'CIVILIZATION' 'LOCATION' ,'TERM']

        print(list( set(sub_list) | set(obj_list)))

        for row in tqdm(dataset[:]['sentence']) :
            try :
                pororo_to_sentence = list(set(ner(row)))
            except :
                continue
            pororo_to_sentence = [(i[0],i[1])  for i in pororo_to_sentence if i[1] in list(set(sub_list) | set(obj_list))]
            
            for idx , item in enumerate(combinations(pororo_to_sentence, 2)) :
                if (item[0][1] in sub_list) and (item[1][1] in obj_list)  :
                    re_idx.append(start)
                    sentence.append(row)
                    subject_entity.append(item[0][0])
                    object_entity.append(item[1][0])
                    label.append(100)
                    start = start +1

        out_dataset = pd.DataFrame(
            {
                "id": re_idx,
                "sentence": sentence,
                "subject_entity": subject_entity,
                "object_entity": object_entity,
                "label": label,
            }
        )

        return out_dataset

    def tokenized_dataset(
        self, dataset: pd.DataFrame, tokenizer: AutoTokenizer
    ) -> torch.Tensor:
        """tokenizer에 따라 sentence를 tokenizing 합니다."""
        concat_entity = []
        for e01, e02 in zip(dataset["subject_entity"], dataset["object_entity"]):
            temp = ""
            temp = e01 + "[SEP]" + e02
            concat_entity.append(temp)

        tokenized_sentences = tokenizer(
            concat_entity,
            list(dataset["sentence"]),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256,
            add_special_tokens=True,
        )
        return tokenized_sentences


    def get_test_dataset(self) -> RE_Dataset:
        """데이터셋을 Trainer에 넣을 수 있도록 처리하여 리턴합니다.

        Args:
            data_path (str): 가져올 데이터의 주소입니다.
            tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

        Returns:
            pd.DataFrame: _description_
        """
        pd_dataset = pd.read_csv(self.data_path)
        dataset = self.preprocessing_dataset(pd_dataset[:])
        pre_sentence = dataset['sentence']
        pre_sub_word = dataset['subject_entity']
        pre_obj_word = dataset['object_entity']

        dataset_label = list(map(int, dataset["label"].values))
        dataset_id = dataset["id"]

        # tokenizing dataset
        dataset_tokens = self.tokenized_dataset(dataset, self.tokenizer)
        # make dataset for pytorch.
        dataset = RE_Dataset(dataset_tokens, dataset_label)
        return dataset_id, dataset, dataset_label , pre_sentence , pre_sub_word , pre_obj_word

def load_dataloader(
    dataloder_type: str, data_path: pathlib.Path, tokenizer: AutoTokenizer
):
    """_summary_

    Args:
        dataloder_type (str) : 가져올 dataloder 클래스 입니다 config 확인
        load_data (pathlib.Path): 가져올 데이터의 주소입니다.
        tokenizer (AutoTokenizer): 데이터를 토큰화할 토크나이저입니다.

    Returns:
        dataloader class : dataloader_type에 맞는 class 반환 합니다
    """
    dataloader_config = {
        "BaseDataLoader": BaseDataLoader(data_path, tokenizer),
    }
    return dataloader_config[dataloder_type]

In [179]:
def inference(model, tokenized_sent, batch_size, device):
    """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
    """
    dataloader = DataLoader(tokenized_sent, batch_size, shuffle=False)
    model.eval()

    output_pred = []
    output_prob = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data["input_ids"].to(device),
                attention_mask=data["attention_mask"].to(device),
                token_type_ids=data["token_type_ids"].to(device),
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
        output_prob.append(prob)

    return (
        np.concatenate(output_pred).tolist(),
        np.concatenate(output_prob, axis=0).tolist(),
    )

In [180]:
#모델 로드
import torch

model_name = 'klue/roberta-large'
load_model = '../dataset/best_model/klue_roberta-large/1.6.0'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(load_model)
model.to(device)
print('--load model --- ')

--load model --- 


In [181]:
test_id, test_dataset, test_label , pre_sentence, pre_sub_word, pre_obj_word = load_dataloader(
    'BaseDataLoader', '../dataset/train/train.csv', tokenizer
).get_test_dataset()


  0%|          | 3/32470 [00:00<28:04, 19.27it/s]

['PERSON', 'LOCATION']


100%|██████████| 32470/32470 [39:21<00:00, 13.75it/s]  


In [182]:
pred_answer, output_prob = inference(
        model, test_dataset, 32, device
    )  # model에서 class 추론

pred_answer = num_to_label(pred_answer)  # 숫자로 된 class를 원래 문자열 라벨로 변환.

## make csv file with predicted answer
#########################################################

# 아래 directory와 columns의 형태는 지켜주시기 바랍니다.
output = pd.DataFrame(
    {
        "id": test_id,
        "sentence" : pre_sentence,
        "sub_word" : pre_sub_word,
        "obj_word" : pre_obj_word,
        "pred_label": pred_answer,
        "probs": output_prob,
    }
)

100%|██████████| 286/286 [02:01<00:00,  2.35it/s]


In [183]:
pd.set_option('display.max_rows', 10)
print(len(output))

mask1 = (output.pred_label == 'per:place_of_residence')
output_mask1 = output.loc[mask1,:]
print(len(output_mask1))

9144
63


In [184]:
output_mask1["max_prob"] = output_mask1["probs"].apply(lambda x : max(x))
output_mask1

<ipython-input-184-87c9ab3a51e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_mask1["max_prob"] = output_mask1["probs"].apply(lambda x : max(x))


,id,sentence,sub_word,obj_word,pred_label,probs,max_prob
68,68,"시는 앞서 지난 9일 지역구 국회의원인 민홍철(김해갑), 김정호(김해을) 의원을 시...",김정호,김해을,per:place_of_residence,"[0.05283449590206146, 0.03590802475810051, 0.0...",0.465385
154,154,"더불어민주당 이상민(대전 유성을), 김종민(충남 논산·계룡·금산), 어기구(당진)의...",김종민,금산,per:place_of_residence,"[0.25116652250289917, 0.01187527272850275, 0.0...",0.420248
314,314,"전남도의회 정옥님 의원(곡성, 더불어민주당)은 11월 1일 장석웅 전남도교육감 집무...",정옥님,곡성,per:place_of_residence,"[0.31468915939331055, 0.09378596395254135, 0.0...",0.318454
412,412,"이날 토론회에는 경상북도의회 장경식 의장과 이철우 경북도지사, 임종식 경북교육감을 ...",최교일,영주,per:place_of_residence,"[0.16095922887325287, 0.004335222765803337, 0....",0.555226
843,843,보테가 베네타는 1966년 이탈리아 비첸자에서 미켈레 타데이(Michele Tadd...,미켈레 타데이,비첸자,per:place_of_residence,"[0.2092004269361496, 0.0073759183287620544, 0....",0.370963
...,...,...,...,...,...,...,...
8591,8591,이날 에너지의 날 홍보대사로 위촉된 안산출신 개그맨 강성범과 정승환은 행사에 참여한...,정승환,안산,per:place_of_residence,"[0.2927241623401642, 0.002696910873055458, 0.0...",0.300719
8973,8973,(1419년 9월 10일). 그래서 새로운 부르고뉴 공작 선량공 필리프(Philip...,선량공 필리프,부르고뉴,per:place_of_residence,"[0.013532575219869614, 0.011670448817312717, 0...",0.500479
8974,8974,"한편, 김헌창(金憲昌)이 822년 3월에 반란을 일으켜, 웅진(熊津)을 수도로 하고...",김헌창,웅진,per:place_of_residence,"[0.16509409248828888, 0.004141645506024361, 0....",0.541764
9128,9128,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,김재원,상주,per:place_of_residence,"[0.30551740527153015, 0.0031655915081501007, 0...",0.399989


In [196]:
mask2 = (output_mask1.max_prob >= 0.4)
output_mask2 = output_mask1.loc[mask2,:]
len(output_mask2)

46

In [197]:
output_mask2

,id,sentence,sub_word,obj_word,pred_label,probs,max_prob
68,68,"시는 앞서 지난 9일 지역구 국회의원인 민홍철(김해갑), 김정호(김해을) 의원을 시...",김정호,김해을,per:place_of_residence,"[0.05283449590206146, 0.03590802475810051, 0.0...",0.465385
154,154,"더불어민주당 이상민(대전 유성을), 김종민(충남 논산·계룡·금산), 어기구(당진)의...",김종민,금산,per:place_of_residence,"[0.25116652250289917, 0.01187527272850275, 0.0...",0.420248
412,412,"이날 토론회에는 경상북도의회 장경식 의장과 이철우 경북도지사, 임종식 경북교육감을 ...",최교일,영주,per:place_of_residence,"[0.16095922887325287, 0.004335222765803337, 0....",0.555226
1534,1534,민주평화당 출신 민생당 의원들의 심상치 않은 거취관련 분위기가 감지되고 있는 가운데...,조배숙,익산 을,per:place_of_residence,"[0.3247184157371521, 0.015921633690595627, 0.0...",0.412893
1894,1894,중화민국 유신정부는 1940년 3월 30일 난징을 수도로 하는 왕징웨이 정권에 중화...,왕징웨이,난징,per:place_of_residence,"[0.16552063822746277, 0.010306784883141518, 0....",0.483546
...,...,...,...,...,...,...,...
8230,8230,인상파 기법의 창시자이자 거장으로 불리는 클로드 모네는 파리에서 태어나 1883년에...,클로드 모네,지베르니,per:place_of_residence,"[0.058040548115968704, 0.002752303145825863, 0...",0.425273
8585,8585,백제 부흥운동을 지원하기 위한 원병 파병문제로 어머니 사이메이 천황과 형 나카노오에...,나카노오에,지쿠시,per:place_of_residence,"[0.04510025307536125, 0.0033492003567516804, 0...",0.601308
8973,8973,(1419년 9월 10일). 그래서 새로운 부르고뉴 공작 선량공 필리프(Philip...,선량공 필리프,부르고뉴,per:place_of_residence,"[0.013532575219869614, 0.011670448817312717, 0...",0.500479
8974,8974,"한편, 김헌창(金憲昌)이 822년 3월에 반란을 일으켜, 웅진(熊津)을 수도로 하고...",김헌창,웅진,per:place_of_residence,"[0.16509409248828888, 0.004141645506024361, 0....",0.541764


In [198]:
def assign_index(row):
    subj_start = row["sentence"].find(row["sub_word"])
    obj_start = row["sentence"].find(row["obj_word"])
    return (subj_start, subj_start + len(row["sub_word"]), obj_start, obj_start + len(row["obj_word"]))

In [199]:
output_mask2[["subj_start_idx", "subj_end_idx", "obj_start_idx", "obj_end_idx"]] = output_mask2.apply(lambda x: pd.Series(assign_index(x)), axis=1)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [200]:
output_mask2[['subj_type' , 'obj_type', 'source']] = ['PER' , 'LOC' , 'wikipedia']

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
<ipython-input-200-25dee2f3ab6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_mask2[['subj_type' , 'obj_type', 'source']] = ['PER' , 'LOC' , 'wikipedia']
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1758: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [201]:
output_mask2["subject_entity"] = output_mask2.apply(
    lambda x:"{" + f"'word': '{x.sub_word}', 'start_idx': {x.subj_start_idx}, 'end_ids': {x.subj_end_idx}, 'type': '{x.subj_type}'" + "}", axis=1)
output_mask2["object_entity"] = output_mask2.apply(
    lambda x:"{" + f"'word': '{x.obj_word}', 'start_idx': {x.obj_start_idx}, 'end_ids': {x.obj_end_idx}, 'type': '{x.obj_type}'" + "}", axis=1)

<ipython-input-201-1d213267823f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_mask2["subject_entity"] = output_mask2.apply(
<ipython-input-201-1d213267823f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_mask2["object_entity"] = output_mask2.apply(


In [202]:
output_mask2[:5]

,id,sentence,sub_word,obj_word,pred_label,probs,max_prob,subj_start_idx,subj_end_idx,obj_start_idx,obj_end_idx,subj_type,obj_type,source,subject_entity,object_entity
68,68,"시는 앞서 지난 9일 지역구 국회의원인 민홍철(김해갑), 김정호(김해을) 의원을 시...",김정호,김해을,per:place_of_residence,"[0.05283449590206146, 0.03590802475810051, 0.0...",0.465385,32,35,36,39,PER,LOC,wikipedia,"{'word': '김정호', 'start_idx': 32, 'end_ids': 35...","{'word': '김해을', 'start_idx': 36, 'end_ids': 39..."
154,154,"더불어민주당 이상민(대전 유성을), 김종민(충남 논산·계룡·금산), 어기구(당진)의...",김종민,금산,per:place_of_residence,"[0.25116652250289917, 0.01187527272850275, 0.0...",0.420248,20,23,33,35,PER,LOC,wikipedia,"{'word': '김종민', 'start_idx': 20, 'end_ids': 23...","{'word': '금산', 'start_idx': 33, 'end_ids': 35,..."
412,412,"이날 토론회에는 경상북도의회 장경식 의장과 이철우 경북도지사, 임종식 경북교육감을 ...",최교일,영주,per:place_of_residence,"[0.16095922887325287, 0.004335222765803337, 0....",0.555226,146,149,153,155,PER,LOC,wikipedia,"{'word': '최교일', 'start_idx': 146, 'end_ids': 1...","{'word': '영주', 'start_idx': 153, 'end_ids': 15..."
1534,1534,민주평화당 출신 민생당 의원들의 심상치 않은 거취관련 분위기가 감지되고 있는 가운데...,조배숙,익산 을,per:place_of_residence,"[0.3247184157371521, 0.015921633690595627, 0.0...",0.412893,47,50,51,55,PER,LOC,wikipedia,"{'word': '조배숙', 'start_idx': 47, 'end_ids': 50...","{'word': '익산 을', 'start_idx': 51, 'end_ids': 5..."
1894,1894,중화민국 유신정부는 1940년 3월 30일 난징을 수도로 하는 왕징웨이 정권에 중화...,왕징웨이,난징,per:place_of_residence,"[0.16552063822746277, 0.010306784883141518, 0....",0.483546,35,39,24,26,PER,LOC,wikipedia,"{'word': '왕징웨이', 'start_idx': 35, 'end_ids': 3...","{'word': '난징', 'start_idx': 24, 'end_ids': 26,..."


In [203]:
#pred_label	probs	max_prob
output_mask2 = output_mask2.drop(["subj_start_idx", "subj_end_idx", "subj_type", "obj_start_idx", "obj_end_idx", "obj_type", "sub_word", "obj_word" , "probs" , "max_prob"], axis=1)
output_mask2 = output_mask2.rename(columns={'pred_label':'label'})

In [204]:
output_mask2

,id,sentence,label,source,subject_entity,object_entity
68,68,"시는 앞서 지난 9일 지역구 국회의원인 민홍철(김해갑), 김정호(김해을) 의원을 시...",per:place_of_residence,wikipedia,"{'word': '김정호', 'start_idx': 32, 'end_ids': 35...","{'word': '김해을', 'start_idx': 36, 'end_ids': 39..."
154,154,"더불어민주당 이상민(대전 유성을), 김종민(충남 논산·계룡·금산), 어기구(당진)의...",per:place_of_residence,wikipedia,"{'word': '김종민', 'start_idx': 20, 'end_ids': 23...","{'word': '금산', 'start_idx': 33, 'end_ids': 35,..."
412,412,"이날 토론회에는 경상북도의회 장경식 의장과 이철우 경북도지사, 임종식 경북교육감을 ...",per:place_of_residence,wikipedia,"{'word': '최교일', 'start_idx': 146, 'end_ids': 1...","{'word': '영주', 'start_idx': 153, 'end_ids': 15..."
1534,1534,민주평화당 출신 민생당 의원들의 심상치 않은 거취관련 분위기가 감지되고 있는 가운데...,per:place_of_residence,wikipedia,"{'word': '조배숙', 'start_idx': 47, 'end_ids': 50...","{'word': '익산 을', 'start_idx': 51, 'end_ids': 5..."
1894,1894,중화민국 유신정부는 1940년 3월 30일 난징을 수도로 하는 왕징웨이 정권에 중화...,per:place_of_residence,wikipedia,"{'word': '왕징웨이', 'start_idx': 35, 'end_ids': 3...","{'word': '난징', 'start_idx': 24, 'end_ids': 26,..."
...,...,...,...,...,...,...
8230,8230,인상파 기법의 창시자이자 거장으로 불리는 클로드 모네는 파리에서 태어나 1883년에...,per:place_of_residence,wikipedia,"{'word': '클로드 모네', 'start_idx': 23, 'end_ids':...","{'word': '지베르니', 'start_idx': 56, 'end_ids': 6..."
8585,8585,백제 부흥운동을 지원하기 위한 원병 파병문제로 어머니 사이메이 천황과 형 나카노오에...,per:place_of_residence,wikipedia,"{'word': '나카노오에', 'start_idx': 41, 'end_ids': ...","{'word': '지쿠시', 'start_idx': 48, 'end_ids': 51..."
8973,8973,(1419년 9월 10일). 그래서 새로운 부르고뉴 공작 선량공 필리프(Philip...,per:place_of_residence,wikipedia,"{'word': '선량공 필리프', 'start_idx': 32, 'end_ids'...","{'word': '부르고뉴', 'start_idx': 24, 'end_ids': 2..."
8974,8974,"한편, 김헌창(金憲昌)이 822년 3월에 반란을 일으켜, 웅진(熊津)을 수도로 하고...",per:place_of_residence,wikipedia,"{'word': '김헌창', 'start_idx': 4, 'end_ids': 7, ...","{'word': '웅진', 'start_idx': 32, 'end_ids': 34,..."


In [172]:
output_mask2.to_csv('./per:place_of_residence.csv', encoding='utf-8', index=False)